In [ ]:
import pandas as pd
import plotly.express as px
import json
from urllib.request import urlopen

## NYC Squirrel Census 2018
[Data Source](https://data.cityofnewyork.us/Environment/2018-Central-Park-Squirrel-Census-Squirrel-Data/vfnx-vebw/about_data)  
[scatter_mapbox Documentation](https://plotly.github.io/plotly.py-docs/generated/plotly.express.scatter_mapbox.html)

In [ ]:
# Squirrel Dataset
nyc_squirrels = pd.read_csv("https://raw.githubusercontent.com/BaroqueObama/Fun-Datasets/main/2018_Central_Park_Squirrel_Census_-_Squirrel_Data.csv")
# Remove squirrel sightings without detailed descriptions
nyc_squirrels = nyc_squirrels.dropna(subset=["Specific Location"])
nyc_squirrels = nyc_squirrels.dropna(subset=["Other Activities"])

# Make the mapbox plot (Documentation: https://plotly.github.io/plotly.py-docs/generated/plotly.express.scatter_mapbox.html)
fig = px.scatter_mapbox(nyc_squirrels, 
                        lat="Y", lon="X", 
                        hover_name="Unique Squirrel ID", hover_data=["Primary Fur Color", "Age", "Specific Location", "Other Activities"], 
                        color_discrete_sequence=["fuchsia"], title="Central Park Squirrels")
# Map style. Others include 'open-street-map', 'white-bg', 'carto-positron', 'carto-darkmatter', 'stamen- terrain', 'stamen-toner', 'stamen-watercolor'
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# Sets center of the map in coordinates
fig.update_mapboxes(center=dict(lat=40.7826, lon=-73.9656))
# Sets bearing of the map to make central park horizontal.
fig.update_mapboxes(bearing=28.912-90)
# Sets zoom of the map. Bigger number, more zoom.
fig.update_mapboxes(zoom=13)
fig.show()

## 2020 General Election results by precinct
[Data Source](https://github.com/TheUpshot/presidential-precinct-map-2020)  
[NYTimes Map](https://www.nytimes.com/interactive/2021/upshot/2020-election-map.html)  
[choropleth_mapbox Documentation](https://plotly.github.io/plotly.py-docs/generated/plotly.express.choropleth.html)  

In [ ]:
# GeoJSON data on precincts and election results
# You can use "bayarea.geojson" for faster results instead.
with open("cal.geojson") as response:
    precincts = json.load(response)
df = pd.json_normalize(precincts['features'])
display(df)

In [ ]:
fig = px.choropleth_mapbox(df, geojson=precincts, 
                    locations="properties.GEOID", 
                    featureidkey="properties.GEOID",
                    color="properties.pct_dem_lead",
                    hover_name="properties.pct_dem_lead",
                    hover_data={"properties.votes_dem": True, "properties.votes_rep": True, "properties.votes_total": True, "properties.pct_dem_lead": False, "properties.GEOID": False},
                    opacity=0.6,
                    color_continuous_scale="RdBu",
                    range_color=[-75,75]
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_traces(marker_line_width=0)
fig.update_mapboxes(center=dict(lat=37.3347, lon=-122.0466))
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_mapboxes(zoom=12)
fig.show()

## Activity 1: County Prison Admissions
[Data Source](https://github.com/TheUpshot/prison-admissions/blob/master/county-prison-admissions.csv)  
[NY Times Article](https://www.nytimes.com/2016/09/02/upshot/new-geography-of-prisons.html)  
  
![Sample NYTimes Map](images/sample_prison.png)  

In [ ]:
# Prison Data
df = pd.read_csv("county-prison.csv", dtype={"fips": str})
# Creates new column that fills 2014 NaN values with 2013 values
df["admitsPerCapita"] = df["admitsPer10k2014"].fillna(df["admitsPer10k2013"])

# Contains GeoJSON data with polygon shapes for all US counties. 
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

display(df)
display(counties)

In [ ]:
# Creates choropleth plot.
# Documentation: https://plotly.github.io/plotly.py-docs/generated/plotly.express.choropleth.html
fig = px.choropleth_mapbox(df, # Data frame with the prison data
                           geojson=, # TODO: assing the GeoJSON data (which contain polygon shapes for each US county)
                           locations=, # TODO: column name from df with location IDs geojson will reference. We will be using fips 
                           featureidkey=, # TODO: path to fips field in geojson which will identify which polygon to use
                           color=, # TODO: which column value in df to adjust the color of the county by. You can choose freely but recommend "admitsPerCapita"
                           hover_name=, # TODO: which column value in df to set as the name
                           hover_data=, # TODwhich column values in df to display when hovering over county
                           opacity=, # TODO: Set opacity of counties between 0 and 1
                           color_continuous_scale=, # TODO: Set your color scheme. You can find them here: https://plotly.com/python/builtin-colorscales/#:~:text=Join%20now.-,Built%2DIn%20Sequential%20Color%20scales,-A%20collection%20of
                           range_color= # TODO: Can set color range if you want to.
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_mapboxes(center=dict(lat=37.3347, lon=-122.0466)) # TODO: Set the starting center coordinates of your map. 
fig.update_traces(marker_line_width=0) # TODO: Can adjust line width if you want.
fig.update_layout(mapbox_style="carto-positron") # TODO: Select your mapbox style: 'open-street-map', 'white-bg', 'carto-positron', 'carto-darkmatter', 'stamen- terrain', 'stamen-toner', 'stamen-watercolor'
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_mapboxes(zoom=12) # TODO: Set your zoom
fig.show()

<details>
  <summary><h2>Solution/Potential Method (Click to Expand)</h2></summary>

```python
# Creates choropleth plot.
# Documentation: https://plotly.github.io/plotly.py-docs/generated/plotly.express.choropleth.html
fig = px.choropleth_mapbox(df, # Data frame with the prison data
                           geojson=counties, # TODO: assing the GeoJSON data (which contain polygon shapes for each US county)
                           locations="fips", # TODO: column name from df with location IDs geojson will reference. We will be using fips 
                           featureidkey="id", # TODO: path to fips field in geojson which will identify which polygon to use
                           color="admitsPerCapita", # TODO: which column value in df to adjust the color of the county by. You can choose freely but recommend "admitsPerCapita"
                           hover_name="county", # TODO: which column value in df to set as the name
                           hover_data=["admitsPer10k2014"], # TODO: which column values in df to display when hovering over county
                           opacity=0.8, # TODO: Set opacity of counties between 0 and 1
                           color_continuous_scale="deep", # TODO: Set your color scheme. You can find them here: https://plotly.com/python/builtin-colorscales/#:~:text=Join%20now.-,Built%2DIn%20Sequential%20Color%20scales,-A%20collection%20of
                           range_color=[0,100] # TODO: Can set color range if you want to.
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_traces(marker_line_width=0) # TODO: Can adjust line width if you want.
fig.update_mapboxes(center=dict(lat=39.8283, lon=-98.5795)) # TODO: Set the starting center coordinates of your map. 
fig.update_layout(mapbox_style="carto-positron") # TODO: Select your mapbox style: 'open-street-map', 'white-bg', 'carto-positron', 'carto-darkmatter', 'stamen- terrain', 'stamen-toner', 'stamen-watercolor'
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_mapboxes(zoom=3) # TODO: Set your zoom
fig.show()
```
</details>

## Activity 2: Use your Google timeline location data.
Go to [takeout](https://takeout.google.com/) for your google account and export your Location History (Timeline) data in JSON format (if you have location history turned on).  
You can then plot your location history using scatter_mapbox